I accidentally deleted everything. Here's my attempt to reconstruct one of the figures I made in the past

In [1]:
import os
import numpy as np 
from scipy import stats
from sedflow import obs as Obs
from sedflow import train as Train

In [2]:
from IPython.display import IFrame
# --- plotting --- 
import corner as DFM
import matplotlib as mpl
import matplotlib.pyplot as plt
#mpl.use('PDF')
#mpl.rcParams['text.usetex'] = True
mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['axes.linewidth'] = 1.5
mpl.rcParams['axes.xmargin'] = 1
mpl.rcParams['xtick.labelsize'] = 'x-large'
mpl.rcParams['xtick.major.size'] = 5
mpl.rcParams['xtick.major.width'] = 1.5
mpl.rcParams['ytick.labelsize'] = 'x-large'
mpl.rcParams['ytick.major.size'] = 5
mpl.rcParams['ytick.major.width'] = 1.5
mpl.rcParams['legend.frameon'] = False

In [3]:
import torch
import torch.nn as nn 
import torch.nn.functional as F 
from sbi import utils as Ut
from sbi import inference as Inference

In [4]:
def dirichlet_transform(tt): 
    ''' warped manifold transformation as specified in Betancourt (2013).
    This function transforms samples from a uniform distribution to a
    Dirichlet distribution .

    x_i = (\prod\limits_{k=1}^{i-1} z_k) * f 

    f = 1 - z_i         for i < m
    f = 1               for i = m 

    Parameters
    ----------
    tt : array_like[N,m-1]
        N samples drawn from a (m-1)-dimensional uniform distribution 

    Returns
    -------
    tt_d : array_like[N,m]
        N transformed samples drawn from a m-dimensional dirichlet
        distribution 

    Reference
    ---------
    * Betancourt(2013) - https://arxiv.org/pdf/1010.3436.pdf
    '''
    tt_d = np.empty(tt.shape[:-1]+(tt.shape[1]+1,)) 

    tt_d[...,0] = 1. - tt[...,0]
    for i in range(1,tt.shape[1]): 
        tt_d[...,i] = np.prod(tt[...,:i], axis=-1) * (1. - tt[...,i]) 
    tt_d[...,-1] = np.prod(tt, axis=-1) 
    return tt_d 

# Comparison of ANPE vs MCMC posterior for single galaxy

In [5]:
prior_low = [7, 0., 0., 0., 0., 1e-2, np.log10(4.5e-5), np.log10(4.5e-5), 0, 0., -2.]
prior_high = [12.5, 1., 1., 1., 1., 13.27, np.log10(1.5e-2), np.log10(1.5e-2), 3., 3., 1.]
lower_bounds = torch.tensor(prior_low)
upper_bounds = torch.tensor(prior_high)

prior = Ut.BoxUniform(low=lower_bounds, high=upper_bounds, device='cpu')

fanpe = '/scratch/network/chhahn/sedflow/anpe_thetaunt_magsigz.toy.%s.pt' % arch

anpe = Inference.SNPE(prior=prior, density_estimator=Ut.posterior_nn('maf', hidden_features=500, num_transforms=15), device='cpu')
anpe.append_simulations(
    torch.as_tensor(_x_test.astype(np.float32)), 
    torch.as_tensor(y_test.astype(np.float32)))

p_x_y_estimator = anpe._build_neural_net(torch.as_tensor(_x_test.astype(np.float32)), torch.as_tensor(y_test.astype(np.float32)))
p_x_y_estimator.load_state_dict(torch.load(fanpe))

anpe._x_shape = Ut.x_shape_from_simulation(torch.as_tensor(y_test.astype(np.float32)))

hatp_x_y = anpe.build_posterior(p_x_y_estimator)

# Load NSA data and MCMC posteriors

In [6]:
nsa, _ = Obs.NSA()

flux_nsa = nsa['NMGY'][:,2:]
ivar_nsa = nsa['NMGY_IVAR'][:,2:]

zred_nsa = nsa['Z']

mags_nsa = Train.flux2mag(flux_nsa)
sigs_nsa = Train.sigma_flux2mag(ivar_nsa**-0.5, flux_nsa)

y_nsa = np.concatenate([mags_nsa, sigs_nsa, zred_nsa[:,None]], axis=1)

/home/chhahn/projects/SEDflow/src/sedflow/train.py:106: RuntimeWarning: invalid value encountered in log10
  return 22.5 - 2.5 * np.log10(flux)


In [7]:
arch = '500x15.2'
anpe_samples = np.load('/scratch/network/chhahn/sedflow/anpe_thetaunt_magsigz.toy.%s.nsa0of34.samples.npy' % arch)

In [8]:
from provabgs import models as Models

nsa_filters = Train.photometry_bands()

m_sps = Models.NMF(burst=True, emulator=True)

/home/chhahn/projects/provabgs/src/provabgs/models.py:23: UserWarning: import error with fsps; only use emulators
  warnings.warn('import error with fsps; only use emulators')


input parameters : logmstar, beta1_sfh, beta2_sfh, beta3_sfh, beta4_sfh, fburst, tburst, gamma1_zh, gamma2_zh, dust1, dust2, dust_index


In [9]:
#i_nsa = 25
i_nsa = 10
print(nsa['NSAID'][i_nsa])

26


In [10]:
c_light = 2.998e18
jansky_cgs = 1e-23

lambda_sdss = np.array([3543., 4770., 6231., 7625., 9134.])

flux_nsa_conv = flux_nsa[i_nsa] * 1e-9 * 1e17 * c_light / lambda_sdss**2 * (3631. * jansky_cgs) # convert to 10^-17 ergs/s/cm^2/Ang
ivar_nsa_conv = ivar_nsa[i_nsa] * (1e-9 * 1e17 * c_light / lambda_sdss**2 * (3631. * jansky_cgs))**-2.


In [11]:
dir_mcmc = '/scratch/network/chhahn/arcoiris/sedflow/mcmc_nsa_redo/'

_anpe = anpe_samples[i_nsa]#np.array(hatp_x_y.sample((10000,), x=torch.as_tensor(y_nsa[i_nsa,:]), show_progress_bars=True))
anpe = np.zeros((_anpe.shape[0], _anpe.shape[1]+1))
anpe[:,0] = _anpe[:,0]
# transform back to dirichlet space
anpe[:,1:5] = dirichlet_transform(_anpe[:,1:4])
anpe[:,5:] = _anpe[:,4:]    

_mcmc = np.load(os.path.join(dir_mcmc, 'mcmc.nsa.%i.npy' % i_nsa))
_mcmc = Train.flatten_chain(_mcmc[4000:])
mcmc = np.zeros((_mcmc.shape[0], _mcmc.shape[1]+1))
mcmc[:,0] = _mcmc[:,0]
# transform back to dirichlet space
mcmc[:,1:5] = dirichlet_transform(_mcmc[:,1:4])
mcmc[:,5:] = _mcmc[:,4:]    

In [12]:
fspecs_anpe, fluxes_anpe = [], []
for _tt in anpe[::10]: 
    tt = _tt.copy()
    tt[7:9] = 10**tt[7:9]
    w_anpe, f_anpe = m_sps.sed(tt, zred_nsa[i_nsa])
    _maggies = Train.SED_to_maggies(w_anpe, f_anpe, filters=nsa_filters)
    fspecs_anpe.append(f_anpe)
    fluxes_anpe.append(np.array(_maggies.as_array()[0].tolist()) * 1e9)

dflux = flux_nsa[i_nsa] - np.array(fluxes_anpe)
log_prob_anpe = -0.5 * np.sum(dflux**2 * ivar_nsa[i_nsa])  

mags_anpe = Train.flux2mag(fluxes_anpe[np.argmax(log_prob_anpe)])
f_anpe = fspecs_anpe[np.argmax(log_prob_anpe)]

print(anpe[::10][np.argmax(log_prob_anpe)])
print(np.max(log_prob_anpe))

[10.41071224  0.06619018  0.36098975  0.53917652  0.03364352  0.58307612
  7.36263418 -3.73200631 -4.04672289  0.39671102  1.57147384  0.53952688]
-963294.4800906982


In [13]:
fspecs_mcmc, fluxes_mcmc = [], []
for _tt in mcmc[::100]: 
    tt = _tt.copy()
    tt[7:9] = 10**tt[7:9]
    w_mcmc, f_mcmc = m_sps.sed(tt, zred_nsa[i_nsa])
    _maggies = Train.SED_to_maggies(w_mcmc, f_mcmc, filters=nsa_filters)
    fspecs_mcmc.append(f_mcmc)
    fluxes_mcmc.append(np.array(_maggies.as_array()[0].tolist()) * 1e9)

dflux = flux_nsa[i_nsa] - np.array(fluxes_mcmc)
log_prob_mcmc = -0.5 * np.sum(dflux**2 * ivar_nsa[i_nsa])  

mags_mcmc = Train.flux2mag(fluxes_mcmc[np.argmax(log_prob_mcmc)])
f_mcmc = fspecs_mcmc[np.argmax(log_prob_mcmc)]
print(mcmc[::100][np.argmax(log_prob_mcmc)])
print(np.max(log_prob_mcmc))

[10.12742446  0.52636253  0.3357474   0.08540599  0.05248409  0.63927353
 10.07684303 -2.91644395 -3.04253988  2.34544149  1.06465423  0.24564591]
-5412.213072225691


In [14]:
print(mags_nsa[i_nsa])
print(mags_anpe)
print(mags_mcmc)

[19.214428 17.8736   17.14103  16.76466  16.484669]
[19.25678641 17.85193371 17.14798585 16.74395196 16.44345872]
[19.2883002  17.87364139 17.15784455 16.75957111 16.43966526]


In [15]:
_low = [7, 0., 0., 0., 0., 0., 1e-2, np.log10(4.5e-5), np.log10(4.5e-5), 0, 0., -2.]
_high = [12.5, 1., 1., 1., 1., 1., 13.27, np.log10(1.5e-2), np.log10(1.5e-2), 3., 3., 1.]
theta_range = [(l, h) for l, h in zip(_low, _high)]

In [16]:
# read in photometric bandpass filters 
from speclite import filters as specFilter
filter_response = specFilter.load_filters('sdss2010-*')
wave_eff = [filter_response[i].effective_wavelength.value for i in range(len(filter_response))]

In [17]:
theta_lbls = [r'$\log M_*$', r"$\beta_1$", r"$\beta_2$", r"$\beta_3$", r"$\beta_4$", r'$f_{\rm burst}$', r'$t_{\rm burst}$', r'$\log \gamma_1$', r'$\log \gamma_2$', r'$\tau_1$', r'$\tau_2$', r'$n_{\rm dust}$']
mpl.use('PDF')
mpl.rcParams['text.usetex'] = True

ndim = len(theta_lbls)

fig = plt.figure(figsize=(15, 18))

gs0 = fig.add_gridspec(nrows=ndim, ncols=ndim, top=0.95, bottom=0.28)
for yi in range(ndim):
    for xi in range(ndim):
        sub = fig.add_subplot(gs0[yi, xi])

_fig = DFM.corner(mcmc, color='k', levels=[0.68, 0.95], range=theta_range,
                  plot_density=False, plot_datapoints=False, hist_kwargs={'density': True}, fig=fig)
_ = DFM.corner(anpe, color='C0', levels=[0.68, 0.95], range=theta_range,
               plot_density=False, plot_datapoints=False, hist_kwargs={'density': True}, 
               labels=theta_lbls, label_kwargs={'fontsize': 25}, fig=fig)


# ndim = int(np.sqrt(len(fig.axes)))
axes = np.array(fig.axes).reshape((ndim, ndim))

ax = axes[2, ndim-4]
ax.fill_between([], [], [], color='k', label='MCMC posterior')
ax.fill_between([], [], [], color='C0', label='ANPE posterior')
ax.legend(handletextpad=0.2, markerscale=10, fontsize=25)

axes = np.array(fig.axes).reshape((ndim, ndim))
for yi in range(1, ndim):
    ax = axes[yi, 0]
    ax.set_ylabel(theta_lbls[yi], fontsize=20, labelpad=30)
    ax.yaxis.set_label_coords(-0.6, 0.5)
for xi in range(ndim): 
    ax = axes[-1, xi]
    ax.set_xlabel(theta_lbls[xi], fontsize=20, labelpad=30)
    ax.xaxis.set_label_coords(0.5, -0.55)
for xi in range(ndim): 
    ax = axes[xi,xi]
    ax.set_xlim(theta_range[xi])

gs1 = fig.add_gridspec(nrows=1, ncols=5, top=0.2, bottom=0.05)
for ii in range(5):
    sub = fig.add_subplot(gs1[0,ii])

    sub.plot(w_anpe, f_anpe, c='C0', lw=1, label='ANPE best-fit')
    sub.plot(w_mcmc, f_mcmc, c='k', lw=1, ls=':', label='MCMC best-fit')
    #for i in range(len(filter_response)): 
    #    #sub.plot(filter_response[i].wavelength, 15 + 30 * filter_response[i].response, ls='--') 
    #    sub.text(filter_response[i].effective_wavelength.value, 1.1 * 30 * np.max(filter_response[i].response), ['u', 'g', 'r', 'i', 'z'][i], fontsize=20, color='C%i' % i)

    sub.errorbar(lambda_sdss, flux_nsa_conv, yerr=ivar_nsa_conv**-0.5, fmt='.C3', markersize=20, elinewidth=4, label='NSA Photometry')
    if ii == 4: sub.legend(loc='upper right', fontsize=18, handletextpad=0)
    if ii == 2: sub.set_xlabel('wavelength [$A$]', fontsize=20) 
    #sub.set_xlim(3e3, 1e4)
    sub.set_xlim(lambda_sdss[ii]-500, lambda_sdss[ii]+500)
    if ii == 0: sub.set_ylabel('flux [$10^{-17} erg/s/cm^2/A$]', fontsize=20, labelpad=15) 
    else: sub.set_yticklabels([])
    if ii == 1: sub.set_xticks([4500, 4750, 5000])
    #if ii == 1: sub.set_xticks([4500, 4750, 5000])
    sub.set_ylim(10., 65.)
    sub.set_title('$%s$' % ['u', 'g', 'r', 'i', 'z'][ii], fontsize=20)

fig.savefig('../paper1/figs/_corner.pdf', bbox_inches='tight')
IFrame("../paper1/figs/_corner.pdf", width=600, height=600)